In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import shutil
from tqdm.notebook import tqdm
import numpy as np
import json
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_curve, auc
)
import seaborn as sns
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

drive_dataset_path = '/content/drive/MyDrive/DocuForge/dataset'
local_dataset_path = '/content/dataset'

# Function to copy dataset with progress
def copy_dataset(src, dst):
    if not os.path.exists(dst):
        os.makedirs(dst)

    for root, dirs, files in os.walk(src):
        # Recreate directory structure
        rel_path = os.path.relpath(root, src)
        dest_dir = os.path.join(dst, rel_path)
        os.makedirs(dest_dir, exist_ok=True)

        # Copy files with progress bar
        for file in tqdm(files, desc=f"Copying {rel_path}", unit="file"):
            src_file = os.path.join(root, file)
            dest_file = os.path.join(dest_dir, file)
            if not os.path.exists(dest_file):
                shutil.copy2(src_file, dest_file)

# Run it
copy_dataset(drive_dataset_path, local_dataset_path)

print("✅ Dataset copied successfully!")

Mounted at /content/drive


Copying .: 0file [00:00, ?file/s]

Copying test: 0file [00:00, ?file/s]

Copying test/authentic:   0%|          | 0/300 [00:00<?, ?file/s]

Copying test/forged:   0%|          | 0/300 [00:00<?, ?file/s]

...

Copying train/authentic:   0%|          | 0/1400 [00:00<?, ?file/s]

Copying val: 0file [00:00, ?file/s]

Copying val/authentic:   0%|          | 0/300 [00:00<?, ?file/s]

Copying val/forged:   0%|          | 0/300 [00:00<?, ?file/s]

✅ Dataset copied successfully!


In [8]:
data_path = '/content/dataset/'

IMG_SIZE = 224  # ResNet50 default input size

train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Directories inside Google Drive
train_dir = data_path + 'train'
val_dir = data_path + 'val'
test_dir = data_path + 'test'

# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=val_transforms)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True, num_workers=2)

print(f"Classes: {train_dataset.classes}")
print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")

Classes: ['authentic', 'forged']
Train: 2800 | Val: 600 | Test: 600


In [9]:
param_grid = {
    'learning_rate': [0.001, 0.01, 0.0001],
    'batch_size': [16, 32, 64],
    'optimizer': ['adam', 'sgd', 'adamw'],
    'weight_decay': [1e-4, 1e-3, 0],
    'dropout_rate': [0.3, 0.5, 0.7],
    'hidden_units': [128, 256, 512]
}

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet50
model = models.resnet50(weights='IMAGENET1K_V2')

# Freeze earlier layers (optional fine-tuning)
for param in model.parameters():
    param.requires_grad = False

# Replace final FC layer
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 1),
)

model = model.to(device)

In [11]:
for name, param in model.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters: {trainable_params}/{total_params}")

Trainable parameters: 22587905/24032833


In [12]:
criterion = nn.BCEWithLogitsLoss()

# Separate learning rates for pretrained vs new layers
fc_params = list(model.fc.parameters())
pretrained_params = [p for n, p in model.named_parameters() if p.requires_grad and "fc" not in n]

optimizer = torch.optim.AdamW([
    {"params": pretrained_params, "lr": 1e-5},  # smaller LR for pretrained
    {"params": fc_params, "lr": 1e-4}           # higher LR for new FC
], weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

In [13]:
EPOCHS = 15
SAVE_DIR = "saved_models"
os.makedirs(SAVE_DIR, exist_ok=True)

best_val_acc = 0.0

train_losses, val_losses = [], []
train_accs, val_accs = [], []

for epoch in range(EPOCHS):
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, labels = imgs.to(device), labels.float().unsqueeze(1).to(device)
        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_losses.append(train_loss / len(train_loader))
    train_accs.append(train_acc)

    # -----------------------------
    # Validation phase
    # -----------------------------
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = (torch.sigmoid(outputs) > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    val_losses.append(val_loss / len(val_loader))
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Acc={train_acc:.3f} | Val Acc={val_acc:.3f} | Train Loss={train_loss/len(train_loader):.3f} | Val Loss={val_loss/len(val_loader):.3f}")

    # -----------------------------
    # Save current epoch model
    # -----------------------------
    model_path = os.path.join(SAVE_DIR, f"model_epoch_{epoch+1}.pth")
    torch.save(model.state_dict(), model_path)

    # -----------------------------
    # Save best model
    # -----------------------------
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_path = os.path.join(SAVE_DIR, "best_model.pth")
        torch.save(model.state_dict(), best_model_path)
        print(f"🏆 Best model updated (Val Acc={val_acc:.3f})")

print("✅ Training complete.")

Epoch 1/15:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch 1/15 | Train Acc=0.726 | Val Acc=0.767 | Train Loss=0.597 | Val Loss=0.565
🏆 Best model updated (Val Acc=0.767)


Epoch 2/15:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch 2/15 | Train Acc=0.844 | Val Acc=0.868 | Train Loss=0.408 | Val Loss=0.357
🏆 Best model updated (Val Acc=0.868)


Epoch 3/15:   0%|          | 0/44 [00:00<?, ?it/s]

...

Epoch 13/15 | Train Acc=0.877 | Val Acc=0.885 | Train Loss=0.303 | Val Loss=0.291
🏆 Best model updated (Val Acc=0.885)


Epoch 14/15:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch 14/15 | Train Acc=0.877 | Val Acc=0.890 | Train Loss=0.307 | Val Loss=0.288
🏆 Best model updated (Val Acc=0.890)


Epoch 15/15:   0%|          | 0/44 [00:00<?, ?it/s]

Epoch 15/15 | Train Acc=0.879 | Val Acc=0.890 | Train Loss=0.298 | Val Loss=0.288
✅ Training complete.


In [15]:
def evaluate_and_save(model, test_loader, criterion, device, save_dir="evaluation_results"):
    """
    Evaluate the model on test data and save all results (plots + metrics).
    """

    os.makedirs(save_dir, exist_ok=True)

    model.eval()
    test_loss, correct, total = 0.0, 0, 0
    all_labels, all_preds, all_probs = [], [], []

    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc="Evaluating"):
            imgs, labels = imgs.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(imgs)

            # Loss on raw logits
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Probabilities for metrics
            probs = torch.sigmoid(outputs).cpu().numpy().flatten()
            preds = (probs > 0.5).astype(int)
            labels_np = labels.cpu().numpy().flatten()

            all_probs.extend(probs)
            all_preds.extend(preds)
            all_labels.extend(labels_np)

            correct += (preds == labels_np).sum().item()
            total += labels_np.shape[0]


    test_loss /= len(test_loader)
    test_acc = correct / total

    print(f"\n🧪 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

    # -----------------------------------
    # Classification Report
    # -----------------------------------
    report = classification_report(all_labels, all_preds, target_names=["Authentic", "Forged"], output_dict=True)
    print("\n📊 Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=["Authentic", "Forged"]))

    report_path = os.path.join(save_dir, "classification_report.txt")
    with open(report_path, "w") as f:
        f.write(classification_report(all_labels, all_preds, target_names=["Authentic", "Forged"]))
    print(f"📝 Classification report saved to {report_path}")

    # -----------------------------------
    # Confusion Matrix
    # -----------------------------------
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=["Authentic", "Forged"],
                yticklabels=["Authentic", "Forged"])
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    cm_path = os.path.join(save_dir, "confusion_matrix.png")
    plt.savefig(cm_path, bbox_inches="tight")
    plt.close()
    print(f"🖼️ Confusion matrix saved to {cm_path}")

    # -----------------------------------
    # ROC Curve and AUC
    # -----------------------------------
    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(5, 4))
    plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC Curve (AUC = {roc_auc:.3f})")
    plt.plot([0, 1], [0, 1], color="gray", lw=1, linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve for Forgery Detection")
    plt.legend(loc="lower right")

    roc_path = os.path.join(save_dir, "roc_curve.png")
    plt.savefig(roc_path, bbox_inches="tight")
    plt.close()
    print(f"📉 ROC curve saved to {roc_path}")

    # -----------------------------------
    # Save numeric results
    # -----------------------------------
    results = {
        "test_loss": float(test_loss),
        "test_accuracy": float(test_acc),
        "roc_auc": float(roc_auc),
        "precision_authentic": report["Authentic"]["precision"],
        "recall_authentic": report["Authentic"]["recall"],
        "f1_authentic": report["Authentic"]["f1-score"],
        "precision_forged": report["Forged"]["precision"],
        "recall_forged": report["Forged"]["recall"],
        "f1_forged": report["Forged"]["f1-score"]
    }

    results_path = os.path.join(save_dir, "metrics.json")
    with open(results_path, "w") as f:
        json.dump(results, f, indent=4)
    print(f"📦 Metrics saved to {results_path}")

    print("\n✅ Evaluation complete. All results saved in:", os.path.abspath(save_dir))

    return test_loss, test_acc, roc_auc

In [16]:
# Load best model
model.load_state_dict(torch.load("saved_models/best_model.pth"))
model.to(device)

# Run detailed evaluation
test_loss, test_acc, roc_auc = evaluate_and_save(model, test_loader, criterion, device)

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]


🧪 Test Loss: 0.3662 | Test Accuracy: 0.8533

📊 Classification Report:
              precision    recall  f1-score   support

   Authentic       0.77      1.00      0.87       300
      Forged       1.00      0.71      0.83       300

    accuracy                           0.85       600
   macro avg       0.88      0.85      0.85       600
weighted avg       0.88      0.85      0.85       600

📝 Classification report saved to evaluation_results/classification_report.txt
🖼️ Confusion matrix saved to evaluation_results/confusion_matrix.png
📉 ROC curve saved to evaluation_results/roc_curve.png
📦 Metrics saved to evaluation_results/metrics.json

✅ Evaluation complete. All results saved in: /content/evaluation_results
